# Checkpointing Workflow Runs

A `Checkpoint` is a snapshot taken during a `Workflow` run that can be inspected and also be used as a starting point in future `Workflow` runs. These checkpoints can become quite helpful when debugging a `Workflow`. For example, if your `Workflow` has many steps and you want to test out the later steps (likely after making some modifications to them), you can save a lot of time by running from the appropriate `Checkpoint` to skip the execution of those earlier steps.

What get's stored in a `Checkpoint`?

- `Checkpoint` objects are centered around the last completed step of the workflow run. They contain the name of the last completed step, that step's input event as well as it's output event, and finally a snapshot of the run's `Context`.

When do `Checkpoint`'s happen?

- When enabled, `Checkpoints` are automatically created and stored in the `Workflow.checkpoints` attribute after every completed step of the `Workflow`.

In the rest of this notebook, we demonstrate:

1. How to enable checkpoints
2. Filter the stored checkpoints, and
3. Finally run from a chosen checkpoint

## Define a Workflow

In [ ]:
import os

api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
from llama_index.core.workflow import (
    Workflow,
    step,
    StartEvent,
    StopEvent,
    Event,
    Context,
)
from llama_index.llms.openai import OpenAI


class JokeEvent(Event):
    joke: str


class JokeFlow(Workflow):
    llm = OpenAI(api_key=api_key)

    @step
    async def generate_joke(self, ev: StartEvent) -> JokeEvent:
        topic = ev.topic

        prompt = f"Write your best joke about {topic}."
        response = await self.llm.acomplete(prompt)
        return JokeEvent(joke=str(response))

    @step
    async def critique_joke(self, ev: JokeEvent) -> StopEvent:
        joke = ev.joke

        prompt = f"Give a thorough analysis and critique of the following joke: {joke}"
        response = await self.llm.acomplete(prompt)
        return StopEvent(result=str(response))

### Running With Checkpointing Disabled (Default)

By default, automatic checkpointing is disabled. However, this can be enabled for any run via the `store_checkpoints` parameter.

In [ ]:
from llama_index.core.workflow.checkpointer import WorkflowCheckpointer

In [ ]:
# instantiate Jokeflow
workflow = JokeFlow()
wflow_ckptr = WorkflowCheckpointer(workflow=workflow)

In [ ]:
handler = wflow_ckptr.run(
    topic="chemistry",
    store_checkpoints=False,
)
await handler

'This joke plays on the double meaning of the word "nitrates," which can refer to both a chemical compound and a form of payment for services rendered. The humor lies in the unexpected twist of associating a scientific term with a financial concept.\n\nOne strength of this joke is its clever wordplay and the unexpected connection between chemistry and economics. It requires the listener to make a mental leap between two seemingly unrelated concepts, which can make the punchline more satisfying.\n\nHowever, one potential weakness of this joke is that it relies heavily on a specific knowledge of chemistry and may not be easily understood by those who are not familiar with the subject. Additionally, the humor may be considered somewhat niche and may not appeal to a broad audience.\n\nOverall, while this joke may be appreciated by those with a background in chemistry, its limited appeal and reliance on niche knowledge may limit its effectiveness as a joke for a general audience.'

In [ ]:
wflow_ckptr.checkpoints

{'27598cfb-fccf-4ef1-98bc-1efadcc062c8': [Checkpoint(id_='840b57c3-a291-455f-9ddd-0edbb0f1bb15', last_completed_step='generate_joke', input_event=StartEvent(), output_event=JokeEvent(joke="Why do chemists like nitrates so much?\n\nBecause they're cheaper than day rates!"), ctx_state={'globals': {}, 'streaming_queue': '[]', 'queues': {'_done': '[]', 'critique_joke': '[]', 'generate_joke': '[]'}, 'stepwise': False, 'events_buffer': {}, 'accepted_events': [('generate_joke', 'StartEvent'), ('critique_joke', 'JokeEvent')], 'broker_log': ['{"__is_pydantic": true, "value": {"_data": {"topic": "chemistry", "store_checkpoints": false}}, "qualified_name": "llama_index.core.workflow.events.StartEvent"}'], 'is_running': True}),
  Checkpoint(id_='c0283a8e-c7b2-4989-aeef-c8d4a957cf3e', last_completed_step='critique_joke', input_event=JokeEvent(joke="Why do chemists like nitrates so much?\n\nBecause they're cheaper than day rates!"), output_event=StopEvent(result='This joke plays on the double meanin

As we can see there are no entries in our `Workflow.checkpoints` attribute.

### Running the Workflow With Checkpointing Enabled

In [ ]:
# run the workflow again, but this time with checkpointing enabled
handler = wflow_ckptr.run(topic="math", store_checkpoints=True)
await handler

'Analysis:\nThis joke plays on the mathematical concept of the equal sign, which is used to show that two quantities are the same. The humor comes from personifying the equal sign and attributing human emotions and characteristics to it. By saying that the equal sign is humble because it knows it is not less than or greater than anyone else, the joke cleverly ties in the mathematical meaning of equality with the idea of humility.\n\nCritique:\nOverall, this joke is clever and well-crafted. It effectively combines a mathematical concept with a play on words to create humor. The punchline is unexpected and plays on the double meaning of "less than" and "greater than" in both a mathematical and a personal sense. However, some may find the joke to be a bit too niche or intellectual, as it relies on a basic understanding of math to fully appreciate the humor. Additionally, the joke may not be as universally relatable as other types of humor, as not everyone may find math jokes funny. Overal

In [ ]:
wflow_ckptr.checkpoints

{'27598cfb-fccf-4ef1-98bc-1efadcc062c8': [Checkpoint(id_='840b57c3-a291-455f-9ddd-0edbb0f1bb15', last_completed_step='generate_joke', input_event=StartEvent(), output_event=JokeEvent(joke="Why do chemists like nitrates so much?\n\nBecause they're cheaper than day rates!"), ctx_state={'globals': {}, 'streaming_queue': '[]', 'queues': {'_done': '[]', 'critique_joke': '[]', 'generate_joke': '[]'}, 'stepwise': False, 'events_buffer': {}, 'accepted_events': [('generate_joke', 'StartEvent'), ('critique_joke', 'JokeEvent')], 'broker_log': ['{"__is_pydantic": true, "value": {"_data": {"topic": "chemistry", "store_checkpoints": false}}, "qualified_name": "llama_index.core.workflow.events.StartEvent"}'], 'is_running': True}),
  Checkpoint(id_='c0283a8e-c7b2-4989-aeef-c8d4a957cf3e', last_completed_step='critique_joke', input_event=JokeEvent(joke="Why do chemists like nitrates so much?\n\nBecause they're cheaper than day rates!"), output_event=StopEvent(result='This joke plays on the double meanin

After enabling checkpointing with specifying `store_checkpoints=True` in the `run()` call, we see that there is a new entry within the `checkpoints` dict. Checkpoints are organized by `run_id`'s and each invocation of a `run()` or `run_from()` (demo'ed later in this notebook) kicks off a new run with it's unique `run_id`.

We can see here that there are 3 stored checkpoints after this first run. The first of these checkpoints are created just before the emission of the `StartEvent`. This can be thought of there being a fictitious "startup" step that is completed (and thus checkpointed), which contains a null input `Event` and whose output event is the `StartEvent`. The remaining two of these checkpoints are made at the completion of each of the two steps—`generate_joke` and `critique_joke`—in this Workflow. 

In [ ]:
for run_id, ckpts in wflow_ckptr.checkpoints.items():
    print(f"Run: {run_id} has {len(ckpts)} stored checkpoints")

Run: 27598cfb-fccf-4ef1-98bc-1efadcc062c8 has 2 stored checkpoints
Run: df3781d7-824a-48a4-8c07-56c8600a0c6b has 2 stored checkpoints


### Filtering the Checkpoints

With checkpointing enabled, every run of a `Workflow`, will create a new entry in the `Workflow.checkpoints` dict attribute. To assist in navigating through these store checkpoints, we provide the `Workflow.filter_checkpoints()` method.

Before showcasing how to filter through checkpoints using this method, let's first make things a bit more interesting by executing the workflow a few more times. Here, we'll run the workflow for two more additional topics. Since each entire run will result in 3 stored checkpoints each, we should have a total of 9 checkpoints after executing the below cell. 

In [ ]:
additional_topics = ["biology", "history"]

for topic in additional_topics:
    handler = wflow_ckptr.run(topic=topic)
    await handler

In [ ]:
for run_id, ckpts in wflow_ckptr.checkpoints.items():
    print(f"Run: {run_id} has {len(ckpts)} stored checkpoints")

Run: 27598cfb-fccf-4ef1-98bc-1efadcc062c8 has 2 stored checkpoints
Run: df3781d7-824a-48a4-8c07-56c8600a0c6b has 2 stored checkpoints
Run: 7f099be8-6ed4-4ef0-92c4-4bc0daa93afb has 2 stored checkpoints
Run: 5099b0b0-0135-4579-9736-2b8f2fd324cc has 2 stored checkpoints


At this point, we can filter by:

- The name of the last completed step by speciying the param `last_completed_step`
- The event type of the last completed step's output event by specifying `output_event_type`
- Similarly, the event type of the last completed step's input event by specifying `input_event_type`

Specifying multiple of these filters will be combined by the "AND" operator.

In [ ]:
# Filter by the name of last completed step
checkpoints_right_after_generate_joke_step = wflow_ckptr.filter_checkpoints(
    last_completed_step="generate_joke",
    run_id=list(wflow_ckptr.checkpoints.keys())[1],
)

# checkpoint ids
[ckpt for ckpt in checkpoints_right_after_generate_joke_step]

[Checkpoint(id_='1cbfd7b5-9b70-4963-be84-e4387eab4ada', last_completed_step='generate_joke', input_event=StartEvent(), output_event=JokeEvent(joke="Why was the equal sign so humble?\n\nBecause he knew he wasn't less than or greater than anyone else."), ctx_state={'globals': {}, 'streaming_queue': '[]', 'queues': {'_done': '[]', 'critique_joke': '[]', 'generate_joke': '[]'}, 'stepwise': False, 'events_buffer': {}, 'accepted_events': [('generate_joke', 'StartEvent'), ('critique_joke', 'JokeEvent')], 'broker_log': ['{"__is_pydantic": true, "value": {"_data": {"topic": "math", "store_checkpoints": true}}, "qualified_name": "llama_index.core.workflow.events.StartEvent"}'], 'is_running': True})]

In [ ]:
# Filter by output event StopEvent
checkpoints_that_emit_stop_event = wflow_ckptr.filter_checkpoints(
    output_event_type=StopEvent
)

# checkpoint ids
[ckpt for ckpt in checkpoints_that_emit_stop_event]

[Checkpoint(id_='c0283a8e-c7b2-4989-aeef-c8d4a957cf3e', last_completed_step='critique_joke', input_event=JokeEvent(joke="Why do chemists like nitrates so much?\n\nBecause they're cheaper than day rates!"), output_event=StopEvent(result='This joke plays on the double meaning of the word "nitrates," which can refer to both a chemical compound and a form of payment for services rendered. The humor lies in the unexpected twist of associating a scientific term with a financial concept.\n\nOne strength of this joke is its clever wordplay and the unexpected connection between chemistry and economics. It requires the listener to make a mental leap between two seemingly unrelated concepts, which can make the punchline more satisfying.\n\nHowever, one potential weakness of this joke is that it relies heavily on a specific knowledge of chemistry and may not be easily understood by those who are not familiar with the subject. Additionally, the humor may be considered somewhat niche and may not app

### Re-Run Workflow from a specific checkpoint

To run from a chosen `Checkpoint` we can use the `Workflow.run_from()` method. NOTE that doing so will lead to a new `run` and it's checkpoints if enabled will be stored under the newly assigned `run_id`.

In [ ]:
# can work with a new instance
new_workflow_instance = JokeFlow()
wflow_ckptr.workflow = new_workflow_instance

ckpt = checkpoints_right_after_generate_joke_step[0]
num_runs_from_checkpoint = 2  # to make things interesting

for _ in range(num_runs_from_checkpoint):
    handler = wflow_ckptr.run_from(checkpoint=ckpt)
    await handler

In [ ]:
for run_id, ckpts in wflow_ckptr.checkpoints.items():
    print(f"Run: {run_id} has {len(ckpts)} stored checkpoints")

Run: 27598cfb-fccf-4ef1-98bc-1efadcc062c8 has 2 stored checkpoints
Run: df3781d7-824a-48a4-8c07-56c8600a0c6b has 2 stored checkpoints
Run: 7f099be8-6ed4-4ef0-92c4-4bc0daa93afb has 2 stored checkpoints
Run: 5099b0b0-0135-4579-9736-2b8f2fd324cc has 2 stored checkpoints
Run: 1db86c39-e6a5-4bac-9adb-e7831088f27e has 1 stored checkpoints
Run: f34d0c19-9f44-46b4-8445-1826aa241d06 has 1 stored checkpoints


Since we've executed from the checkpoint that represents the end of "generate_joke" step, there is only one additional checkpoint that gets stored in these "partial" runs.

### Disabling Checkpoints

In [ ]:
wflow_ckptr.checkpoints_config

{'critique_joke': True, 'generate_joke': True}

In [ ]:
wflow_ckptr.disable_checkpoint("critique_joke")

{'critique_joke': False}

In [ ]:
handler = wflow_ckptr.run(topic="art")
await handler

'Analysis:\nThis joke plays on the double meaning of the phrase "brush strokes." In art, brush strokes refer to the marks made by a paintbrush on a canvas, while in a psychological context, "strokes" can refer to positive reinforcement or approval. The joke suggests that the artist went to therapy because they had too many brush strokes, implying that they were seeking validation or approval for their work.\n\nCritique:\n- Clever wordplay: The joke is clever in its use of the double meaning of "brush strokes" to create humor. It requires the listener to make a connection between the two meanings, which can be satisfying for those who appreciate wordplay.\n- Relatable: The joke may resonate with artists or anyone familiar with the creative process, as seeking validation for one\'s work is a common experience.\n- Lack of depth: While the joke is clever, it may be seen as somewhat shallow or predictable. The punchline is fairly straightforward and doesn\'t offer much in terms of complexit

In [ ]:
for run_id, ckpts in wflow_ckptr.checkpoints.items():
    print(f"Run: {run_id} has {[c.last_completed_step for c in ckpts]}")

Run: 27598cfb-fccf-4ef1-98bc-1efadcc062c8 has ['generate_joke', 'critique_joke']
Run: df3781d7-824a-48a4-8c07-56c8600a0c6b has ['generate_joke', 'critique_joke']
Run: 7f099be8-6ed4-4ef0-92c4-4bc0daa93afb has ['generate_joke', 'critique_joke']
Run: 5099b0b0-0135-4579-9736-2b8f2fd324cc has ['generate_joke', 'critique_joke']
Run: 1db86c39-e6a5-4bac-9adb-e7831088f27e has ['critique_joke']
Run: f34d0c19-9f44-46b4-8445-1826aa241d06 has ['critique_joke']
Run: da1f9c18-dd1b-4e85-8bcb-562456d7f7e7 has ['generate_joke']


### Enable Checkpoints

In [ ]:
wflow_ckptr.enable_checkpoint("critique_joke")

{'critique_joke': True}

In [ ]:
handler = wflow_ckptr.run(topic="medicine")
await handler

'Analysis:\nThis joke plays on the double meaning of the phrase "draw blood." In the medical context, "drawing blood" refers to the process of taking a blood sample from a patient for testing. However, in a more literal sense, "drawing blood" can also mean causing someone to bleed, typically through physical injury.\n\nThe joke sets up the expectation that the doctor carries a red pen for medical purposes, but then subverts that expectation by revealing a more humorous and unexpected reason - to draw blood in the literal sense. This unexpected twist is what makes the joke funny.\n\nCritique:\nOverall, this joke is clever and plays on the dual meanings of a common medical term. The punchline is unexpected and adds an element of surprise, which is essential for a successful joke. However, the humor may be a bit dark or morbid for some audiences, as it involves the idea of causing someone to bleed.\n\nAdditionally, the joke relies heavily on wordplay and may not be immediately understood 

In [ ]:
for run_id, ckpts in wflow_ckptr.checkpoints.items():
    print(f"Run: {run_id} has {[c.last_completed_step for c in ckpts]}")

Run: 27598cfb-fccf-4ef1-98bc-1efadcc062c8 has ['generate_joke', 'critique_joke']
Run: df3781d7-824a-48a4-8c07-56c8600a0c6b has ['generate_joke', 'critique_joke']
Run: 7f099be8-6ed4-4ef0-92c4-4bc0daa93afb has ['generate_joke', 'critique_joke']
Run: 5099b0b0-0135-4579-9736-2b8f2fd324cc has ['generate_joke', 'critique_joke']
Run: 1db86c39-e6a5-4bac-9adb-e7831088f27e has ['critique_joke']
Run: f34d0c19-9f44-46b4-8445-1826aa241d06 has ['critique_joke']
Run: da1f9c18-dd1b-4e85-8bcb-562456d7f7e7 has ['generate_joke']
Run: 3ef2b39e-fff1-4d7c-bf75-a70a8143416d has ['generate_joke', 'critique_joke']
